# Data Processing

In [1]:
import pandas as pd
import os
import json
from IPython.display import display
import matplotlib.pyplot as plt
import pyspark

clean_path="../data/clean/"
processing_path="../data/processing/"

# Initial Data
_parameters = pd.read_json(clean_path+"parameters.json")
_countries = pd.read_json(clean_path+"countries.json")
_measuraments = pd.read_json(clean_path+"measurements.json")

display(_parameters)
display(_countries)
display(_measuraments)

ValueError: Trailing data

In [3]:
# Create Spark Context
sc = pyspark.SparkContext('local[*]', appName='SparkActivity')

In [7]:
measurements = sc.textFile(clean_path+"measurements.json")
measurements = measurements.map(lambda i: i['average'])
print(measurements.first())




Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (bcba6b8d2707 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1560, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-7-22900